In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import csv
from urllib.parse import urljoin, urlparse
import os
import re
import time
import urllib.request

from selenium import webdriver
from requests_html import HTMLSession

import shutil # to save the image locally

In [2]:
# Using when pages/images are not dynamically rendered

def get_response(url):
    session = requests.Session()
    response = session.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    #response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    return soup

In [3]:
# Use when pages/images are dynamic. The function uses selenium and renders the page in Firefox webdriver

def get_sel_response(url):
    driver = webdriver.Firefox()
    driver.get(url)
    html = driver.execute_script("return document.documentElement.outerHTML")
    sel_soup = BeautifulSoup(html, "html.parser")
    return sel_soup

In [4]:
'''
Define all the required info to scrape and update in the csv file:
designer_root_url: landing/ homepage of the designer
designer_url : url of the page that needs to be scraped
name : name of the designer
location : location of the designer
'''

designer_root_url = 'https://www.joanlao.com/'
designer_url = 'https://www.joanlao.com/category/collections/'
name = 'Joan Lao'
location = 'Barcelone, Spain'

root_soup = get_response(designer_url)


Code is divided into 4 parts:    
1. Get the list of all the image urls on the designer_url page and store it in a list called product_urls.        
2. Scrape product name, year of designing and image description from each url.   
3. Get all the product images for a specific product
4. Download and save

### Testing each section


In [11]:
# Step 1

products_url = []
#rows = root_soup.findAll(lambda tag: tag.name == 'a' and tag.get('class') == ["thumb  "])
sel_soup = get_response(designer_url)
div = sel_soup.findAll("div", {"class":"grid_1"})

for rows in div:
    url = rows.find("a")['href']
    print(url)
    products_url.append(url)

print(len(products_url))
print(products_url[:5])

https://www.joanlao.com/bespoke-sofa/
https://www.joanlao.com/metallic-gauze-lamp/
https://www.joanlao.com/tissue-shade-lamp/
https://www.joanlao.com/wood-lamp/
https://www.joanlao.com/metal-lamp/
https://www.joanlao.com/easy-chair/
https://www.joanlao.com/peace/
https://www.joanlao.com/sober/
https://www.joanlao.com/eclectic/
https://www.joanlao.com/ethereal/
https://www.joanlao.com/sustainable/
https://www.joanlao.com/simply/
https://www.joanlao.com/collections-basico/
https://www.joanlao.com/collections-changes/
https://www.joanlao.com/collections-vital/
https://www.joanlao.com/collections-difusa/
https://www.joanlao.com/collections-translucida/
https://www.joanlao.com/vital-table/
https://www.joanlao.com/collections-pequenas-arquitecturas/
https://www.joanlao.com/collections-eterna/
https://www.joanlao.com/collections-organic/
https://www.joanlao.com/collections-penumbra/
https://www.joanlao.com/collections-solido/
https://www.joanlao.com/collections-slow/
https://www.joanlao.com/c

In [12]:
# Step 2. 

url = "https://www.joanlao.com/bespoke-sofa/"

soup = get_response(url)
session = requests.Session()
status = session.get(url, headers={'User-Agent': 'Mozilla/5.0'}).status_code
para_text = ""
if status == 200:
    #text = soup.findAll(lambda tag: tag.name == 'table' and tag.get('class') == ['wp-block-table has-fixed-layout'])
    
    #div = soup.find('div', {"class" : "col-lg-3 col-md-4 col-sm-4 col-xs-12 project-desc"})
    product_name = soup.find('h2', {"class":"fancy"}).text
    
    #para_text = div.find("p").text
    
    years = re.findall(r'.*([1-3][0-9]{3})', para_text)
    if len(years) > 0:
        year = years[0]
    else:
        year = 'NA'
    
print(product_name)
print(year)
print(para_text)



BESPOKE SOFA
NA



In [14]:
# Step 3

root_soup = get_response("https://www.joanlao.com/bespoke-sofa/")

#new_soup = get_response("https://www.helenkontouris.com/projects/softscape-pendants-lighting/")
image_list = []
#div = root_soup.find("div", {"class": "col-lg-6 col-md-6 col-sm-6 col-xs-12 project-images"})
images = soup.findAll("img", {"class":"aligncenter"})

for image in images:
    if image.has_attr('src'):
        img_url = image['src']
        print(img_url)
        image_list.append(img_url)
    else:
        continue

    
                        
print(len(image_list))
print(image_list[0:5])


https://www.joanlao.com/wp-content/uploads/design_furniture_collection_bespoke_sofa_joan_lao_12.jpg
https://www.joanlao.com/wp-content/uploads/design_furniture_collection_bespoke_sofa_joan_lao_11.jpg
https://www.joanlao.com/wp-content/uploads/design_furniture_collection_bespoke_sofa_joan_lao_10.jpg
https://www.joanlao.com/wp-content/uploads/design_furniture_collection_bespoke_sofa_joan_lao_4.jpg
https://www.joanlao.com/wp-content/uploads/design_furniture_collection_bespoke_sofa_joan_lao_9.jpg
https://www.joanlao.com/wp-content/uploads/design_furniture_collection_bespoke_sofa_joan_lao_8.jpg
https://www.joanlao.com/wp-content/uploads/design_furniture_collection_bespoke_sofa_joan_lao_7.jpg
7
['https://www.joanlao.com/wp-content/uploads/design_furniture_collection_bespoke_sofa_joan_lao_12.jpg', 'https://www.joanlao.com/wp-content/uploads/design_furniture_collection_bespoke_sofa_joan_lao_11.jpg', 'https://www.joanlao.com/wp-content/uploads/design_furniture_collection_bespoke_sofa_joan_lao_1

## Main code starts

In [5]:
# Get all products urls:
'''
Input : main page of products like news/exhibition/projects etc and response object
Output : list of urls of different product pages
'''

def get_product_urls(root_soup, designer_root_url):
    products_url = []
    div = root_soup.findAll("div", {"class":"grid_1"})

    for rows in div:
        url = rows.find("a")['href']
        print(url)
        products_url.append(url)

    print(len(products_url))   
    return products_url


In [6]:
# Get image urls for each product in a list
'''
Input : designer root url and response object of the specific product page
Output : list of image urls
'''

def get_image_list(soup, designer_root_url):
    image_list = []
    
    images = soup.findAll("img", {"class":"aligncenter"})

    for image in images:
        if image.has_attr('src'):
            img_url = image['src']
            print(img_url)
            image_list.append(img_url)
        else:
            continue

    print(len(image_list))
    print(image_list)
    return image_list

In [7]:
# Writing details to csv

def write_to_csv(details):
    csv_columns = ['Name', 'Website', 'Location', 'Year', 'ProductName', 'Description', 'Total_images']
    data = details
    csv_file = "product_details2.csv"
    
    try:
        with open(csv_file, 'a') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
            for row in data:
                writer.writerow(row)
    except IOError:
        print("I/O error")

    print("Written successfully")

In [8]:
# Download the image

def download_image(img_list, dir_name, product_name):
    if not os.path.isdir(dir_name):
        os.makedirs(dir_name)
        
    for i, image_url in enumerate(img_list):
        print(image_url)
        filename = os.path.join(dir_name, product_name.strip() +'_'+ str(i) +'.jpg')
        print(filename)
        session = requests.Session()
        r = session.get(image_url, headers={'User-Agent': 'Mozilla/5.0'}, stream=True) 
        #r = requests.get(image_url, stream = True)
        print("status code :", r.status_code)
        
        if r.status_code == 200:
            try:
                r.raw.decode_content = True
                with open(filename,'wb') as f:
                    shutil.copyfileobj(r.raw, f)
                    print('Image sucessfully Downloaded: ', filename)
            except UnicodeEncodeError:
                print("Exception caught")
                pass
        else:
            print('Image Couldn\'t be retreived: ', filename)
            
        time.sleep(1) # Sleeping time after scraping each url

In [9]:
# Main Function

def get_product_details(root_soup, designer_url, name, designer_root_url, location):
    
    # Get the product urls
    products_url = get_product_urls(root_soup, designer_root_url)
    
    # Loop over all product urls and get all the required info of product name, year, description and images
    for url in products_url:

        # Establish connection
        soup = get_response(url)
        para_text = ""

        session = requests.Session()
        status = session.get(url, headers={'User-Agent': 'Mozilla/5.0'}).status_code
        print("processing url : ", url, status)
        
        # If the page is valid, get the details
        if status == 200:
            product_name = soup.find('h2', {"class":"fancy"}).text

            years = re.findall(r'.*([1-3][0-9]{3})', para_text)
            if len(years) > 0:
                year = years[0]
            else:
                year = 'NA'
        else:
            continue  

        # Get the image list for the product
        image_list = get_image_list(soup, designer_root_url)
        
        # Save the details to csv
        details = []

        details.append({"Name": name, "Website": url, "Location": location, 
                        'Year': year, 'ProductName': product_name, 'Description': para_text, 
                        'Total_images': len(image_list)})
        
        write_to_csv(details)
        print("written to csv successfully")
        
        #Download the images
        download_image(image_list, name, product_name)
        print("images downloaded successfully")
        time.sleep(1)


In [10]:
#Main function call
get_product_details(root_soup, designer_url, name, designer_root_url, location)

https://www.joanlao.com/bespoke-sofa/
https://www.joanlao.com/metallic-gauze-lamp/
https://www.joanlao.com/tissue-shade-lamp/
https://www.joanlao.com/wood-lamp/
https://www.joanlao.com/metal-lamp/
https://www.joanlao.com/easy-chair/
https://www.joanlao.com/peace/
https://www.joanlao.com/sober/
https://www.joanlao.com/eclectic/
https://www.joanlao.com/ethereal/
https://www.joanlao.com/sustainable/
https://www.joanlao.com/simply/
https://www.joanlao.com/collections-basico/
https://www.joanlao.com/collections-changes/
https://www.joanlao.com/collections-vital/
https://www.joanlao.com/collections-difusa/
https://www.joanlao.com/collections-translucida/
https://www.joanlao.com/vital-table/
https://www.joanlao.com/collections-pequenas-arquitecturas/
https://www.joanlao.com/collections-eterna/
https://www.joanlao.com/collections-organic/
https://www.joanlao.com/collections-penumbra/
https://www.joanlao.com/collections-solido/
https://www.joanlao.com/collections-slow/
https://www.joanlao.com/c

https://www.joanlao.com/wp-content/uploads/design_furniture_collection_tissue_shade_lamp_joan_lao_4.jpg
Joan Lao/TISSUE SHADE LAMP_3.jpg
status code : 200
Image sucessfully Downloaded:  Joan Lao/TISSUE SHADE LAMP_3.jpg
images downloaded successfully
processing url :  https://www.joanlao.com/wood-lamp/ 200
https://www.joanlao.com/wp-content/uploads/design_furniture_collection_wood_lamp_joan_lao_1.jpg
https://www.joanlao.com/wp-content/uploads/design_furniture_collection_wood_lamp_joan_lao_2.jpg
https://www.joanlao.com/wp-content/uploads/design_furniture_collection_wood_lamp_joan_lao_3.jpg
https://www.joanlao.com/wp-content/uploads/design_furniture_collection_wood_lamp_joan_lao_4.jpg
https://www.joanlao.com/wp-content/uploads/design_furniture_collection_wood_lamp_joan_lao_5.jpg
https://www.joanlao.com/wp-content/uploads/design_furniture_collection_wood_lamp_joan_lao_6.jpg
6
['https://www.joanlao.com/wp-content/uploads/design_furniture_collection_wood_lamp_joan_lao_1.jpg', 'https://www.jo

status code : 200
Image sucessfully Downloaded:  Joan Lao/EASY CHAIR_4.jpg
https://www.joanlao.com/wp-content/uploads/design_furniture_collection_easy_chair_joan_lao_7.jpg
Joan Lao/EASY CHAIR_5.jpg
status code : 200
Image sucessfully Downloaded:  Joan Lao/EASY CHAIR_5.jpg
https://www.joanlao.com/wp-content/uploads/design_furniture_collection_easy_chair_joan_lao_8.jpg
Joan Lao/EASY CHAIR_6.jpg
status code : 200
Image sucessfully Downloaded:  Joan Lao/EASY CHAIR_6.jpg
https://www.joanlao.com/wp-content/uploads/design_furniture_collection_easy_chair_joan_lao_4.jpg
Joan Lao/EASY CHAIR_7.jpg
status code : 200
Image sucessfully Downloaded:  Joan Lao/EASY CHAIR_7.jpg
images downloaded successfully
processing url :  https://www.joanlao.com/peace/ 200
https://www.joanlao.com/wp-content/uploads/design_furniture_collection_peace__joan_lao_1.jpg
https://www.joanlao.com/wp-content/uploads/design_furniture_collection_peace__joan_lao_2.jpg
https://www.joanlao.com/wp-content/uploads/design_furniture_c

status code : 200
Image sucessfully Downloaded:  Joan Lao/SUSTAINABLE_0.jpg
https://www.joanlao.com/wp-content/uploads/design_furniture_collection_sustainable_joan_lao_2.jpg
Joan Lao/SUSTAINABLE_1.jpg
status code : 200
Image sucessfully Downloaded:  Joan Lao/SUSTAINABLE_1.jpg
https://www.joanlao.com/wp-content/uploads/design_furniture_collection_sustainable_joan_lao_3.jpg
Joan Lao/SUSTAINABLE_2.jpg
status code : 200
Image sucessfully Downloaded:  Joan Lao/SUSTAINABLE_2.jpg
https://www.joanlao.com/wp-content/uploads/design_furniture_collection_sustainable_joan_lao_4.jpg
Joan Lao/SUSTAINABLE_3.jpg
status code : 200
Image sucessfully Downloaded:  Joan Lao/SUSTAINABLE_3.jpg
https://www.joanlao.com/wp-content/uploads/design_furniture_collection_sustainable_joan_lao_5.jpg
Joan Lao/SUSTAINABLE_4.jpg
status code : 200
Image sucessfully Downloaded:  Joan Lao/SUSTAINABLE_4.jpg
https://www.joanlao.com/wp-content/uploads/design_furniture_collection_sustainable_joan_lao_6.jpg
Joan Lao/SUSTAINABLE_5

https://www.joanlao.com/wp-content/uploads/design_furniture_collection_vital_joan_lao_3.jpg
Joan Lao/VITAL_1.jpg
status code : 200
Image sucessfully Downloaded:  Joan Lao/VITAL_1.jpg
https://www.joanlao.com/wp-content/uploads/design_furniture_collection_vital_joan_lao_1.jpg
Joan Lao/VITAL_2.jpg
status code : 200
Image sucessfully Downloaded:  Joan Lao/VITAL_2.jpg
https://www.joanlao.com/wp-content/uploads/design_furniture_collection_vital_joan_lao_4.jpg
Joan Lao/VITAL_3.jpg
status code : 200
Image sucessfully Downloaded:  Joan Lao/VITAL_3.jpg
https://www.joanlao.com/wp-content/uploads/design_furniture_collection_vital_joan_lao_5.jpg
Joan Lao/VITAL_4.jpg
status code : 200
Image sucessfully Downloaded:  Joan Lao/VITAL_4.jpg
https://www.joanlao.com/wp-content/uploads/design_furniture_collection_vital_joan_lao_2.jpg
Joan Lao/VITAL_5.jpg
status code : 200
Image sucessfully Downloaded:  Joan Lao/VITAL_5.jpg
images downloaded successfully
processing url :  https://www.joanlao.com/collections-

...

status code : 200
Image sucessfully Downloaded:  Joan Lao/CALMA_0.jpg
https://www.joanlao.com/wp-content/uploads/design_furniture_collection_calma_joan_lao_1.jpg
Joan Lao/CALMA_1.jpg
status code : 200
Image sucessfully Downloaded:  Joan Lao/CALMA_1.jpg
https://www.joanlao.com/wp-content/uploads/design_furniture_collection_calma_joan_lao_3.jpg
Joan Lao/CALMA_2.jpg
status code : 200
Image sucessfully Downloaded:  Joan Lao/CALMA_2.jpg
https://www.joanlao.com/wp-content/uploads/design_furniture_collection_calma_joan_lao_4.jpg
Joan Lao/CALMA_3.jpg
status code : 200
Image sucessfully Downloaded:  Joan Lao/CALMA_3.jpg
images downloaded successfully
processing url :  https://www.joanlao.com/vital-sofa/ 200
https://www.joanlao.com/wp-content/uploads/design_furniture_collection_vital_sofa_joan_lao_3.jpg
https://www.joanlao.com/wp-content/uploads/design_furniture_collection_vital_sofa_joan_lao_2.jpg
https://www.joanlao.com/wp-content/uploads/design_furniture_collection_vital_sofa_joan_lao_1.jpg
3

status code : 200
Image sucessfully Downloaded:  Joan Lao/SOFT_0.jpg
https://www.joanlao.com/wp-content/uploads/design_furniture_collection_soft_joan_lao_1.jpg
Joan Lao/SOFT_1.jpg
status code : 200
Image sucessfully Downloaded:  Joan Lao/SOFT_1.jpg
images downloaded successfully
processing url :  https://www.joanlao.com/collections-orbita/ 200
https://www.joanlao.com/wp-content/uploads/design_furniture_collection_orbita_joan_lao_1.jpg
https://www.joanlao.com/wp-content/uploads/design_furniture_collection_orbita_joan_lao_2.jpg
https://www.joanlao.com/wp-content/uploads/design_furniture_collection_orbita_joan_lao_3.jpg
3
['https://www.joanlao.com/wp-content/uploads/design_furniture_collection_orbita_joan_lao_1.jpg', 'https://www.joanlao.com/wp-content/uploads/design_furniture_collection_orbita_joan_lao_2.jpg', 'https://www.joanlao.com/wp-content/uploads/design_furniture_collection_orbita_joan_lao_3.jpg']
Written successfully
written to csv successfully
https://www.joanlao.com/wp-content/

status code : 200
Image sucessfully Downloaded:  Joan Lao/YOUNG_0.jpg
https://www.joanlao.com/wp-content/uploads/design_furniture_alternative_young_joan_lao_2.jpg
Joan Lao/YOUNG_1.jpg
status code : 200
Image sucessfully Downloaded:  Joan Lao/YOUNG_1.jpg
https://www.joanlao.com/wp-content/uploads/design_furniture_alternative_young_joan_lao_3.jpg
Joan Lao/YOUNG_2.jpg
status code : 200
Image sucessfully Downloaded:  Joan Lao/YOUNG_2.jpg
https://www.joanlao.com/wp-content/uploads/design_furniture_alternative_young_joan_lao_5.jpg
Joan Lao/YOUNG_3.jpg
status code : 200
Image sucessfully Downloaded:  Joan Lao/YOUNG_3.jpg
https://www.joanlao.com/wp-content/uploads/design_furniture_alternative_young_joan_lao_8.jpg
Joan Lao/YOUNG_4.jpg
status code : 200
Image sucessfully Downloaded:  Joan Lao/YOUNG_4.jpg
https://www.joanlao.com/wp-content/uploads/design_furniture_alternative_young_joan_lao_7.jpg
Joan Lao/YOUNG_5.jpg
status code : 200
Image sucessfully Downloaded:  Joan Lao/YOUNG_5.jpg
https://ww

status code : 200
Image sucessfully Downloaded:  Joan Lao/CLOSETS_0.jpg
https://www.joanlao.com/wp-content/uploads/design_furniture_collection_closet_joan_lao_2.jpg
Joan Lao/CLOSETS_1.jpg
status code : 200
Image sucessfully Downloaded:  Joan Lao/CLOSETS_1.jpg
https://www.joanlao.com/wp-content/uploads/design_furniture_collection_closet_joan_lao_3.jpg
Joan Lao/CLOSETS_2.jpg
status code : 200
Image sucessfully Downloaded:  Joan Lao/CLOSETS_2.jpg
images downloaded successfully
processing url :  https://www.joanlao.com/collections-bathrooms/ 200
https://www.joanlao.com/wp-content/uploads/design_furniture_collection_bathroom_joan_lao_1.jpg
https://www.joanlao.com/wp-content/uploads/design_furniture_collection_bathroom_joan_lao_2.jpg
https://www.joanlao.com/wp-content/uploads/design_furniture_collection_bathroom_joan_lao_3.jpg
https://www.joanlao.com/wp-content/uploads/design_furniture_collection_bathroom_joan_lao_4.jpg
4
['https://www.joanlao.com/wp-content/uploads/design_furniture_collecti

status code : 200
Image sucessfully Downloaded:  Joan Lao/AFINITAT_0.jpg
https://www.joanlao.com/wp-content/uploads/design_furniture_collection_afinitat_joan_lao_1.jpg
Joan Lao/AFINITAT_1.jpg
status code : 200
Image sucessfully Downloaded:  Joan Lao/AFINITAT_1.jpg
images downloaded successfully
processing url :  https://www.joanlao.com/collections-mistica/ 200
https://www.joanlao.com/wp-content/uploads/design_furniture_collection_mistica_joan_lao_1.jpg
https://www.joanlao.com/wp-content/uploads/design_furniture_collection_mistica_joan_lao_2.jpg
2
['https://www.joanlao.com/wp-content/uploads/design_furniture_collection_mistica_joan_lao_1.jpg', 'https://www.joanlao.com/wp-content/uploads/design_furniture_collection_mistica_joan_lao_2.jpg']
Written successfully
written to csv successfully
https://www.joanlao.com/wp-content/uploads/design_furniture_collection_mistica_joan_lao_1.jpg
Joan Lao/MISTICA_0.jpg
status code : 200
Image sucessfully Downloaded:  Joan Lao/MISTICA_0.jpg
https://www.jo